# Behavior Preprocessing

In [3]:
from grabbit import Layout
import pandas as pd
import numpy as np
import os
from msit import extract_itis, encode_trial_type_sequence, encode_post_error

layout = Layout('../data', '../data/grabbit_config.json')

# Accumulate individual dataframes to be concatenated at the end
tidy_df = []

# Iterate through the two modalities
# func = fmri
for modality in ['eeg', 'func']:
    
    # Extract out behavior filepaths for all subjects
    behavior_files = layout.get(subject='sub-.*', modality=modality, 
                                extensions='.tsv')
    
    # Iterate through each subject's behavior file
    for f in behavior_files:
        print(f)
        
        raw_df = pd.read_csv(f.filename, sep='\t', na_values='n/a')
        
        # Encode subject name
        raw_df['participant_id'] = f.subject
        
        # Encode modality
        if modality == 'func':
            raw_df['modality'] = 'fmri'
        else:
            raw_df['modality'] = 'eeg'
            
        # Extract out the itis
        raw_df = extract_itis(raw_df)
        
        # Remove the fixation trials
        trial_ix = raw_df.trial_type != 'fixation'
        raw_df = raw_df[trial_ix].reset_index(drop=True)
        
        # Encode error trials
        raw_df['error'] = 1 - raw_df.response_accuracy
        
        # Encode post error trials
        raw_df = encode_post_error(raw_df)
        
        # Encode no-response trials
        raw_df['no_response'] = 0
        raw_df.loc[raw_df.chosen_response.isnull(), 'no_response'] = 1
        
        # Encode condition sequence effects
        raw_df = encode_trial_type_sequence(raw_df)
        
        # Encode fast response times (< 200 ms)
        raw_df['fast_rt'] = np.where(raw_df.response_time < 0.2, 1, 0)
        
        # Encode trial numbers
        raw_df['trial'] = np.arange(raw_df.shape[0]) + 1
        
        tidy_df.append(raw_df)
        
tidy_df = pd.concat(tidy_df)

# Select out and order columns
tidy_df = tidy_df[['participant_id', 'modality', 'trial', 'onset', 
                   'duration', 'iti', 'trial_type', 'response_time', 
                   'chosen_response', 'correct_response', 
                   'response_accuracy', 'stimulus', 'error', 'post_error', 
                   'no_response', 'trial_type_sequence', 'fast_rt']]
print(tidy_df.head())

# Save out to file
if not os.path.exists('../data/derivatives/behavior'):
    os.makedirs('../data/derivatives/behavior')
tidy_df.to_csv('../data/derivatives/behavior/group_data.tsv', sep='\t',
               index=False, na_rep='n/a')

File(filename='/autofs/space/cassia_001/users/matt/msit/data/sub-hc001/eeg/sub-hc001_task-msit_events.tsv', modality='eeg', subject='sub-hc001')
File(filename='/autofs/space/cassia_001/users/matt/msit/data/sub-hc002/eeg/sub-hc002_task-msit_events.tsv', modality='eeg', subject='sub-hc002')
File(filename='/autofs/space/cassia_001/users/matt/msit/data/sub-hc003/eeg/sub-hc003_task-msit_events.tsv', modality='eeg', subject='sub-hc003')
File(filename='/autofs/space/cassia_001/users/matt/msit/data/sub-hc004/eeg/sub-hc004_task-msit_events.tsv', modality='eeg', subject='sub-hc004')
File(filename='/autofs/space/cassia_001/users/matt/msit/data/sub-hc005/eeg/sub-hc005_task-msit_events.tsv', modality='eeg', subject='sub-hc005')
File(filename='/autofs/space/cassia_001/users/matt/msit/data/sub-hc006/eeg/sub-hc006_task-msit_events.tsv', modality='eeg', subject='sub-hc006')
File(filename='/autofs/space/cassia_001/users/matt/msit/data/sub-hc007/eeg/sub-hc007_task-msit_events.tsv', modality='eeg', subjec

AttributeError: 'DataFrame' object has no attribute 'trial_type'

# EEG Preprocessing

## Make EEG Preprocessing Derivatives File Structure

EEG isn't standardly supported by the BIDS specification yet. However, we still try to stay reasonably close to the derivatives philosophy currently under development. Thus the preprocessing section (which culminates in cleaned epochs) is stored as an eeg_preprocessing BIDS-derivatives compatible pipeline in the data/derivatives/eeg_preprocessing folder.

In [11]:
make_eeg_prep_derivatives_folder('../data')

ImportError: No module named src

## Mark Bad Channels


In [1]:
from grabbit import Layout

%matplotlib

# extract participant raw filenames
layout = Layout('../data', '../data/grabbit_config.json')
raw_files = layout.get(subject='sub-.*', modality='eeg', 
                       extensions='.fif')

# iterate through participant raw data
for f in raw_files:
    
    manually_mark_bad_channels(f)


Using matplotlib backend: Qt4Agg


## Generate Epochs

### Verify Cleaned Epochs & Average Re-Reference

hc008: maybe need to re-look for eye flutter? really bad  
hc009: alpha?  
hc011: weird?  
hc018: bad  
hc021: need to scrape out frontals  
hc023: remove cp3 for stim  
hc035: weird ramp up in baseline before auto reject (almost 50 trials thrown out)  
pp003: tons of stim locked epochs thrown out by auto reject. probably need to re-mark bad channels and push through.

In [1]:
import json
from grabbit import Layout
from msit import load_epochs, visually_verify_epochs
from mne import set_log_level
import matplotlib.pyplot as plt
import numpy as np

%matplotlib

set_log_level('critical')

layout = Layout('../data', '../data/grabbit_config.json')
subjects = np.array(sorted(layout.get(target='subject', 
                                      modality='eeg',
                                      return_type='id')))
start_ix = np.where(subjects == 'sub-hc006')[0]
end_ix = np.where(subjects == 'sub-pp016')[0]
subjects = subjects[start_ix:end_ix + 1]
pipeline_root = '../data/derivatives/eeg_preprocessing'
    
# iterate through participants
for sub in subjects:
    
    print(sub)
    
    for epo_type in ['stimulus', 'response']:
        print(epo_type)
    
        epochs, ar_epochs = load_epochs(sub, epo_type, layout)
        
        # visually check epochs and mark any persisting bad channels
        ar_epochs = visually_verify_epochs(epochs, ar_epochs)
        
        # average re-reference
        ar_epochs.set_eeg_reference().apply_proj()
        
        # save to file
        ar_epochs.save('%s/%s/autoreject/%s_%s_ar-epo.fif' % (pipeline_root,
                                                              sub, sub,
                                                              epo_type))
        
        # save cleaned butterfly plot
        f, ax = plt.subplots(1, 1, figsize=(20, 12))
        ar_epochs.crop(ar_epochs.times[0] + 1, ar_epochs.times[-1] - 1)
        fig = ar_epochs.average().plot(show=False, spatial_colors=True, 
                                       axes=ax)
        fig.savefig('%s/%s/%s_%s_cleaned_evoked_butterfly.png' % (pipeline_root,
                                                                  sub, sub,
                                                                  epo_type))
        
        # save channel sensors plot
        fig = ar_epochs.plot_sensors(show=True, title=sub, show_names=True)
        fig.savefig('%s/%s/%s_sensors.png' % (pipeline_root, sub, sub))
        plt.close('all')
        
print('Done!')

Using matplotlib backend: Qt4Agg


/autofs/space/cassia_001/users/matt/software/anaconda2.7/lib/python2.7/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


sub-hc006
stimulus


/autofs/space/cassia_001/users/matt/software/anaconda2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/autofs/space/cassia_001/users/matt/software/anaconda2.7/lib/python2.7/site-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-hc007
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-hc008
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-hc009
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-hc010
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-hc011
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y\
OK to move on? (enter y or n):y
sub-hc012
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-hc013
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-hc014
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-hc015
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-hc016
stimulus
OK to move on? (enter y or n):y
response
OK to 

/autofs/space/cassia_001/users/matt/software/anaconda2.7/lib/python2.7/site-packages/autoreject-0.1.dev0-py2.7.egg/autoreject/autoreject.py:640: UserWarning: No bad epochs were found for your data. Returning a copy of the data you wanted to clean. Interpolation may have been done.


OK to move on? (enter y or n):y
sub-pp002
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-pp003
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-pp004
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-pp005
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-pp006
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-pp007
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-pp008
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-pp009
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-pp010
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-pp011
stimulus
OK to move on? (enter y or n):y
response
OK to move on? (enter y or n):y
sub-pp012
stimulus
OK to move on? (enter y or n)